![title_pic](./img/title_page.png)
#### TV & Movie recommendation system using a collaborative and content based filtering approach

# Collaborative Filtering Model

# Project Overview
The main goal for this project is to Develop a hybrid movie/TV recommendation system that combines collaborative filtering and content-based filtering to suggest new content to users. Currently, these techniques are applied independently. Our project aims to harness their combined potential.

**Collaborative Filtering**: Analyzes existing user profiles to discover shared preferences and recommend new content based on similarities.

**Content-Based Filtering**: Suggests new content with similar fearures to the movie/TV show that you input.


# Business Understanding
As streaming platforms pile-up content, users struggle to pinpoint films or shows that align with their tastes. The dubious presence of bias in platform algorithms exacerbates this challenge, making it harder for users to rely on platform recommendations. Biases emerge from factors like skewed user preferences, popularity bias, or even the platform's promotional agenda. As a result, recommended content may not cater to users' unique tastes, negatively affecting the overall user experience.

Streaming platforms stand to gain from implementing an unbiased hybrid recommendation system that blends content-based and collaborative filtering techniques. This approach leverages the best of both methods, increasing reliability and personalization while mitigating biases. The content-based technique analyzes features like genre and content description, while collaborative filtering harnesses the collective trends of user ratings. Together, they forge a powerful recommendation engine, enhancing user satisfaction and overall experience.

# Collaborative Filtering

In the collaborative filtering approach, we focused on user-to-user filtering, comparing users' profiles to identify similarities in movie preferences by looking at the ratings they gave to movies that they have both watched. We started by creating a base model as a benchmark, which predicted the mean rating for each movie. To improve the recommendations, we iterated over two models: an SVD model and an SVDpp model. In the iteration process, we used cross-validation in tandem with GridSearchCV to find the best parameters for our models, optimizing their performance.

This cross-validation and GridSearchCV process helped us fine-tune our models to achieve a Root Mean Square Error (RMSE) score of 0.908. Given that our dataset contains user ratings on a scale of 1 to 5, a 0.908 RMSE score signifies good performance in predicting users' movie ratings. It indicates that, on average, our model's predictions deviate by 0.908 from the actual user ratings. This performance enables more accurate and personalized recommendations for users based on their shared preferences, enhancing the overall user experience and satisfaction.

### Importing All Packages Used

In [6]:
import pandas as pd
import numpy as np
import random


from surprise import Dataset, Reader, accuracy
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.prediction_algorithms import BaselineOnly, SVDpp, SVD


# increasing display to view large descriptions and reviewText
pd.set_option('display.max_colwidth', None)



import warnings
warnings.filterwarnings('ignore')

In [8]:
df_merged = pd.read_csv('./data/collab_merged.csv')
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967643 entries, 0 to 1967642
Data columns (total 8 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   rating       int64 
 1   user_id      object
 2   movie_id     object
 3   reviews      object
 4   genre        object
 5   description  object
 6   title        object
 7   starring     object
dtypes: int64(1), object(7)
memory usage: 120.1+ MB


### Creating a dataframe for evaluating model later

In [9]:
df_meta = df_merged.drop(columns=['reviews'], axis=1)

### Creating dataframe for modeling
- removing all columns that will not be used for collaborative modeling

In [10]:
df = df_merged.drop(columns=['reviews', 'genre', 'description',
                            'title', 'starring'], axis=1)
df.head()

,rating,user_id,movie_id
0,5,A2M1CU2IRZG0K9,0005089549
1,5,A2PANT8U0OJNT4,0005419263
2,5,A3TS9EQCNLU0SM,0005092663
3,3,A2AZ7CE08CWK4F,0005092663
4,4,A2QWTVQ90KYZZP,0005092663


Instantiating Reader()

In [11]:
reader = Reader()
data = Dataset.load_from_df(df[['user_id', 'movie_id', 'rating']], reader)

In [14]:
#splitting data into 80% training and 20% testing data and setting a random state
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

### Benchmarking RMSE
Evaluation: An RMSE value of 0.9352 for the predicted 'rating' (scaled 1 to 5) implies that, on average, the predicted ratings have an error of approximately 0.94. Since the error is less than 1, the models have a relatively decent performance in predicting user ratings. However, further improvements may still be made through hyperparameter tuning and testing more complex models.

In [15]:
# Using BasineLineOnly() as a benchmark to evaluate later model iteration
baselinee = BaselineOnly()
# fitting on training data
baselinee.fit(trainset)
# 
predictions = baselinee.test(testset)
base_pred = accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.9352


## SVD() model

In [18]:
# cross-validating across the entire dataset to evaluate model performance
# across 5 different folds of the dataset vs baselineOnly()
svd_cv = SVD()
cv_svd = cross_validate(svd_cv, data, measures=['RMSE'], n_jobs=-1, verbose=True)

for i in cv_svd.items():
    print(i)

#printing the average RMSE score across the different folds    
print(np.mean(cv_svd['test_rmse']))

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9293  0.9290  0.9297  0.9288  0.9227  0.9279  0.0026  
Fit time          11.86   11.83   12.39   12.09   11.42   11.92   0.32    
Test time         3.64    3.48    3.27    3.04    2.87    3.26    0.28    
('test_rmse', array([0.92928011, 0.92904732, 0.92970563, 0.92883226, 0.92272053]))
('fit_time', (11.855475664138794, 11.831639051437378, 12.393603324890137, 12.08669114112854, 11.42304253578186))
('test_time', (3.636509418487549, 3.4764344692230225, 3.2680933475494385, 3.037435531616211, 2.8677146434783936))
0.9279171697134299


### cross_val Evaluation
- The results from the cross validation show that the SVD model performs slightly better than the baseline model across all datasets.
- Therefore, After gridsearching for the best hyperparameters an SVD() model should perform even better than the baselineOnly() model when fit on the training data.

#### Gridsearching SVD()
- using GridSearchCV to find the best paramaters for an SVD model.

In [25]:
# setting paramaters to gridsearch
params = {'n_factors': [5, 10, 20],
          'n_epochs': [10, 20, 40],
          'reg_all':[0.01, 0.02]
         }

# applying gridsearch using defined parameters
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1, joblib_verbose=10)
g_s_svd.fit(data)

# printing best score and best paramaters from for the model
print(g_s_svd.best_score)
print(g_s_svd.best_params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   25.8s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   47.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  69 out of  90 | elapsed:  2.7min remaining:   48.9s
[Parallel(n_jobs=-1)]: Done  79 out of  90 | elapsed:  3.1min remaining:   25.8s


{'rmse': 0.9159009525623993, 'mae': 0.6250712713648022}
{'rmse': {'n_factors': 5, 'n_epochs': 40, 'reg_all': 0.02}, 'mae': {'n_factors': 5, 'n_epochs': 40, 'reg_all': 0.02}}


[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  3.6min finished


### Instantiantiaging parameters on actual SVD() model

In [26]:
SVD_first = SVD(n_factors=5, n_epochs=40, random_state=42)
SVD_first.fit(trainset)
predictions = SVD_final.test(testset)
SVD_one = accuracy.rmse(predictions)

RMSE: 0.9106


In [27]:
# setting paramaters to gridsearch
params = {'n_factors': [1, 3, 5],
          'n_epochs': [40, 60, 80]
         }

# applying gridsearch using defined parameters
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1, joblib_verbose=10)
g_s_svd.fit(data)

# printing best score and best paramaters from for the model
print(g_s_svd.best_score)
print(g_s_svd.best_params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   37.3s
[Parallel(n_jobs=-1)]: Done  19 out of  45 | elapsed:  1.0min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  24 out of  45 | elapsed:  1.3min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  29 out of  45 | elapsed:  1.6min remaining:   51.6s
[Parallel(n_jobs=-1)]: Done  34 out of  45 | elapsed:  1.7min remaining:   33.5s
[Parallel(n_jobs=-1)]: Done  39 out of  45 | elapsed:  2.0min remaining:   18.1s


{'rmse': 0.9124953022669567, 'mae': 0.6199247463047419}
{'rmse': {'n_factors': 1, 'n_epochs': 40}, 'mae': {'n_factors': 1, 'n_epochs': 60}}


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:  2.2min finished


### Instantiating second SVD() model
- using best params found from gridsearching to save a final SVD() model
- Evaluating performance on testset

In [29]:
SVD_final = SVD(n_factors=1, n_epochs=40, random_state=42)
SVD_final.fit(trainset)
predictions = SVD_final.test(testset)
SVD_last = accuracy.rmse(predictions)

RMSE: 0.9113


## SVD++() model

In [19]:
# cross-validating across the entire dataset to evaluate model performance
# across 5 different folds of the dataset vs baselineOnly()
svd_pp_cv = SVDpp()
cv_svdpp = cross_validate(svd_pp_cv, data, measures=['RMSE'], n_jobs=-1, verbose=True)

for i in cv_svdpp.items():
    print(i)

print(np.mean(cv_svdpp['test_rmse']))

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9168  0.9192  0.9194  0.9205  0.9188  0.9189  0.0012  
Fit time          28.22   28.08   28.13   27.90   27.97   28.06   0.11    
Test time         8.52    8.45    8.55    8.32    8.16    8.40    0.14    
('test_rmse', array([0.91684397, 0.91919397, 0.91938563, 0.92051538, 0.91876144]))
('fit_time', (28.221678018569946, 28.07955265045166, 28.130774974822998, 27.90238857269287, 27.97005558013916))
('test_time', (8.523648023605347, 8.454118490219116, 8.548155307769775, 8.322375774383545, 8.158403158187866))
0.9189400785327425


### cross_val Evaluation
- Again, the results from the cross validation show that the SVD++ model performs slightly better than the baseline model across all datasets.
- Therefore, After gridsearching for the best hyperparameters an SVD() model should perform even better than the baselineOnly() model when fit on the training data.

In [28]:
# setting paramaters to gridsearch
params = {'n_factors': [5, 10, 20],
          'n_epochs': [10, 20, 40],
          'cache_ratings':[True, False]
         }
# applying gridsearch using defined parameters
g_s_svdpp = GridSearchCV(SVDpp,param_grid=params,n_jobs=-1, measures=['RMSE'], joblib_verbose=10)
g_s_svdpp.fit(data)

# printing best score and best paramaters from for the model
print(g_s_svdpp.best_score)
print(g_s_svdpp.best_params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   44.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  69 out of  90 | elapsed:  4.3min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  79 out of  90 | elapsed:  5.3min remaining:   43.8s


{'rmse': 0.9136492420521638}
{'rmse': {'n_factors': 5, 'n_epochs': 20, 'cache_ratings': False}}


[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  6.2min finished


### Instantiating best SVD() model
- using best params found from gridsearching to save a final SVD() model
- Evaluating performance on testset

In [33]:
SVDpp_one = SVDpp(n_factors=5,n_epochs=20, random_state=42)
SVDpp_one.fit(trainset)
predictions = SVDpp_one.test(testset)
SVDpp_last = accuracy.rmse(predictions)

RMSE: 0.9132


## Final Model
Based on the model results It would seem that the first gridsearched SVD() model 'SVD_first' is the best performer with an RMSE: 0.9106. Though with more time to gridsearch even more hyperparamters could lead to a different and better performing model. 
Although with an RMSE of 0.91 for a final model compared to the benchmark of a score of 0.9352, it is not much better than the baseline model and indicates that I may want to do some further feature engineering on my dataset before attempting more hyperparameter tuning.

## Function to return content based on similar users predicted by the final model
To use the final model I created the recommend_movies function to give content recommendations to existing users, based on the trained SVDpp collaborative filtering model. It takes an existing user ID, the model, a  DataFrame like my movie and tv meta dataframe, and an optional parameter N (defaulting to 10) to recommend N movies. By predicting user ratings for unseen movies and sorting them in descending order, the function returns the top N movie details, such as titles and ratings, based on user preferences and similarities with other users.

In [30]:
def recommend_movies(trained_model, movie_df, N=5, user_id=None):
    
    # If no user_id provided, default to a random user_id for evaluations
    if user_id is None:
        all_user_ids = movie_df['user_id'].unique().tolist()
        user_id = random.choice(all_user_ids)

    # Get user's watched movies and all available movies
    user_movies = movie_df[movie_df['user_id'] == user_id]['movie_id'].tolist()
    all_movies = movie_df['movie_id'].tolist()
    
    # Determine the set of unseen movies
    unseen_movies = set(all_movies) - set(user_movies)

    # Predict ratings for unseen movies
    predictions = []
    for movie_id in unseen_movies:
        predicted_rating = trained_model.predict(user_id, movie_id).est
        predictions.append({'movie_id': movie_id, 'predicted_rating': predicted_rating})
    
    # Create a DataFrame with predicted ratings
    predictions_df = pd.DataFrame(predictions)

    # Get the top N movies with the highest predicted ratings
    top_N = predictions_df.sort_values('predicted_rating', ascending=False).head(N)
    top_N_movie_ids = top_N['movie_id'].tolist()

    # Get movie details of the top N movies
    top_N_movies = movie_df[movie_df['movie_id'].isin(top_N_movie_ids)]
    top_N_movies.drop_duplicates(subset=['movie_id'], inplace=True)

    # Merge movie details with predicted ratings
    top_N_ratings = pd.merge(top_N, top_N_movies, on='movie_id')
    
    # Print user information
    num_movies_reviewed = len(user_movies)
    print(f"The user_id for this recommendations is {user_id} and they have reviewed {num_movies_reviewed} different movies and/or TV shows")

    # Return top N movie recommendations without unnecessary columns
    return top_N_ratings.drop(columns=['user_id', 'rating'], axis=1)

In [39]:
recommendations = recommend_movies(SVD_first, df_meta, N=10, user_id='A2Z9MK8HNYT17Y')
recommendations

The user_id for this recommendations is A2Z9MK8HNYT17Y and they have reviewed 5 different movies and/or TV shows


,movie_id,predicted_rating,genre,description,title,starring
0,B00L37J2B2,5.0,Drama,Indian Doctor S3,The Indian Doctor - Series Three,Sanjeev Bhaskar
1,B005FI36OE,5.0,Music Videos & Concerts,"<i>Jackie Evancho: Dream With Me In Concert</i>, marks the solo concert debut of the 10-year-old girl with the extraordinary soprano voice. Already familiar to national audiences from her debut appearances on <i>America's Got Talent</i>, Jackie is the youngest star in the special <i>Hitman Returns: David Foster And Friends</i> concert, performing virtuoso interpretations of Andrew Lloyd Webber's ""Pie Jesu"" and Puccini's ""O Mio Babbino Caro."" Foster co-hosts the special, and serves as music director. Jackie Evancho's concert celebrates the release of her first full-length SYCO/Columbia CD <i>Dream with Me</i>, which covers classical crossover material, and was taped at the spectacular grounds of the magnificent The John and Mable Ringling Museum of Art in Sarasota, Florida. SYCO/Columbia Records released the album, produced by 16-time Grammy Award winner David Foster, on Tuesday, June 14, 2011. <b>Track listing:</b> 1. When You Wish Upon A Star 2. Nella Fantasia 3. O Mio Babbino Caro 4. To Believe 5. Imaginer 6. Lovers 7. Dark Waltz 8. Angel 9. Somewhere 10. Mi Mancherai 11. Ombra Mai Fu 12. All I Ask Of You 13. The Lord's Prayer 14. Dream With Me 15. Nessun Dorma",Jackie Evancho Dream With Me In Concert with Musical Host David Foster,Jackie Evancho
2,B007I8KXJK,5.0,Comedy,"<b>Includes SRM Free Digital Download</b> In a dystopian future, a relentless turf war rages. Two rival gangs feud for control of rural wasteland Frazier Park ('The FP') in the deadly arena of competitive dance-fight video game ""Beat-Beat Revelation."" After hometown hero BTRO is slain on the dance platform by thug leader L Dubba E, his protg younger brother JTRO (Jason Trost) goes into isolation, vowing never to duel again. One year later, The FP is in ruins, and JTRO must find the courage to return and restore order in a ruthless battle for revenge that can only leave one man dancing. From the producers of Paranormal Activity and featuring narration by James Remar (The Warriors), THE FP is a fury of fierce footwork, triumphant montages and neon street wear that Moviefone calls ""a rare ready-made cult hit.""",The FP,Jason Trost
3,B00D3XMETQ,5.0,Comedy,"Jason Priestley stars as Fitz, a morally bankrupt used car salesman. Sure, a career selling pre-owned is pretty sweet, but why stop there when real power's within reach? Question is, how does a guy like Fitz get what he wants, when what he wants is always above the law? Public office, baby.",Call Me Fitz: Complete Third Season,Various Artists
4,B0000VCZK2,5.0,Action & Adventure,"Includes ""Alien,"" ""Aliens,"" ""Alien 3,"" and ""Alien: Resurrection."" The <I>Alien Quadrilogy</I> is a nine-disc boxed set devoted to the four <I>Alien</I> films. Although previously available on DVD as the <I>Alien Legacy</I>, here they have been repackaged with vastly more extras and with upgraded sound and picture. For anyone who hasn't been in hypersleep for the last 25 years, this series needs no introduction, though for the first time each film now comes in both original and ""special edition"" form. <I>Alien</I> (1979) was so perfect it didn't need fixing, and Ridley Scott's 2003 director's cut is fiddling for the sake of fiddling. Watch it once, then return to the majestic, perfectly paced original. Conversely, the special edition of James Cameron's <I>Aliens</I> (1986) is the definitive version, though it's nice to finally have the theatrical cut on DVD for comparison. Most interesting is the alternative <I>Alien 3</I> (1992). This isn't a ""director's cut""--David Fincher refused to have any involvement with this release--but a 1991 work-print that runs 29 minutes longer than the theatrical version, and has now been restored, remastered, and finished off with (unfortunately) cheap new CGI. Still, it's truly fascinating, offerin

In [36]:
df_meta[df_meta['user_id'] == 'A2Z9MK8HNYT17Y']

,rating,user_id,movie_id,genre,description,title,starring
1441458,5,A2Z9MK8HNYT17Y,B00O4CWP7G,Comedy,"Get ready to laugh with Mork & Mindy, the insanely popular Happy Days spin-off that crash-landed onto television in 1978. Fans went wild for this wacky, off-the-wall sitcom featuring a misfit alien who departs from the faraway planet of Ork to come study crazy earthling customs. After touching down near Boulder, Colorado, Mork encounters the beautiful college student Mindy (Pam Dawber). The two strike up a charming and entertaining friendship that is full of good times and unexpected surprises. Revisit the hilarious sitcom that launched Robin Williams career into warp drive with this four season, 15-disc collection.",Mork & Mindy: The Complete Series,Robin Williams
1515098,5,A2Z9MK8HNYT17Y,B00S89IJX0,Comedy,"When widower Mike Brady (Robert Reed) marries a lovely lady named Carol Ann (Florence Henderson), their two families become one. The six children include Carol's Daughters: Marcia (Maureen McCormick), Jan (Eve Plumb) and Cindy (Susan Olsen), as well as Mike's sons: Greg (Barry Williams), Peter (Christopher Knight), and Bobby (Mike Lookinland). Together with their dog Tiger and quirky housekeeper Alice (Ann B. Davis), this wild clan became the Brady Bunch! From the debut episode with Mike and Carol tie the knot, to the heartwarming series finale, this series is filled with family fun, outrageous outfits, and loads of sibling rivalry. Experience all the chaos and laughter of The Brady Bunch with this 117-episode collection!",Brady Bunch: The Complete Series,Florence Henderson
1779429,5,A2Z9MK8HNYT17Y,B000A6T1VS,Unknown,"Will Smith stars as a teenager from inner city Philadelphia who's sent to California to live in with his wealthy relatives in the hopes that they will ""straighten him out and teach him some good old-fashioned values,"" Fresh Prince of Bel-Air: The Complete First Season is a 4-disc collection set featuring lively bonus material plus all 24 Season-One Episodes, from Will's first day at the country club (which one is the fish fork?) to his first day at Bel-Air Prep School (sort of homeboy meets Harvard). When it comes to comedy, Fresh Prince rules! On Season 2, Bel-Air is starting to feel like the hood for Will. His Philly roots run deep, but even a street-smart rapper can wrap his mind around croissants and credit cards and his arms around upscale hotties. Guest stars include Queen Latifah, Milton Berle and Zsa Zsa Gabor.",The Fresh Prince of Bel-Air: The Complete Seasons 1 & 2,Will Smith
1784393,5,A2Z9MK8HNYT17Y,B000BYA4JW,Comedy,"<![CDATA[ Fresh Prince of Bel Air, The: The Complete Third Season (DVD) Season three of The Fresh Prince of Bel-Air finds Will firmly entrenched in the Banks family, having traded his rough-and-tumble Philadelphia youth for posh surroundings in Bel-Air. This year, Will hip-hops over to the same prep school as Carlton and starts scoping out colleges. While Will tries to work his mojo on reluctant Monique, Hilary breaks into show business and learns some hard lessons. But the kids arent the only ones riding lifes highs and lows. Philip (James Avery) jumps into the race for superior court judge, opposing his sleazy but powerful former mentor (Sherman Hemsley, The Jeffersons). And Vivian (Janet Hubert-Whitten) surprises everyone by announcing that shes pregnant! Also appearing in this hilarious and poignant season are Oprah Winfrey, Vanessa Williams (Ugly Betty), comic D.L. Hughley, singer Tom Jones and boxer Riddick Bowe. All 24 episodes are fresh and ready for your enjoyment. <i>The Fresh Prince of Bel-Air</i>'s third season simultaneously provided some of the best and worst episodes of the show's six-year run. Set up as a classic collision of streets and society, the show soon found Will (Will Smith) beginning to adapt a little too well to his upper-crust home. He transfers to the same prep academy as Carlton (Alfonso Ribeiro), peruses Ivy League colleges, and truly starts to believe

### Inferences for first recommendation:
Overall, the recommendations seem to be fairly well-tailored to the user's preferences, with the exception of the music video and concert, and possibly the Action & Adventure movie. The Comedy TV shows are likely to be well-received, but it would be better to focus more on identifying TV show recommendations in the Comedy genre for this user. Likely it's identifying content found around the same timeframe that the user has rated.

In [40]:
recommendations2 = recommend_movies(SVD_first, df_meta, N=10, user_id='A1GF8W7J9526AB')
recommendations2

The user_id for this recommendations is A1GF8W7J9526AB and they have reviewed 11 different movies and/or TV shows


movie_id  predicted_rating                      genre   
0  B0177ZM3MI          4.365437                      Drama  \
1  B0002ERXC2          4.347217                    Unknown   
2  B00GUO3C3W          4.271471                    Unknown   
3  B00JSC8BMU          4.189640     Classics & Silent Film   
4  B0019VM63I          4.186658                Documentary   
5  B008UDXBR8          4.185192  Art House & International   
6  B00114XLUA          4.170017     Classics & Silent Film   
7  B008GY8R98          4.167498                    Unknown   
8  B000AT2OD2          4.162607                    Unknown   
9  B00X7SIALI          4.152341                    Unknown   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [38]:
df_meta[df_meta['user_id'] == 'A1GF8W7J9526AB']

,rating,user_id,movie_id,genre,description,title,starring
237172,3,A1GF8W7J9526AB,6305388539,Unknown,This 90 minute feature film DVD stars Daniel Bernhardt.,Bloodsport 4,Daniel Bernhardt
557007,1,A1GF8W7J9526AB,B000O179DG,Comedy,"A wild college party in a remote cabin will go horribly wrong when Derek Cowley and his friends find themselves terrorized by a blood thirsty werewolf. While fighting for his life, Derek will discover a shocking and horrifying secret about the monster he must kill to survive. Featuring a clever, cunning, and sadistically funny breed of werewolf unlike you've ever seen, Big Bad Wolf is one sexy and terrifying horror film.",Big Bad Wolf,Clint Howard
1110703,5,A1GF8W7J9526AB,B009SCONXQ,Unknown,"A high-octane, globe-spanning thriller with storylines ripped from todays headlines, Strike Back is a one-hour drama focusing on two members of a top-secret British anti-terrorist organization known as Section 20: Michael Stonebridge (Philip Winchester), a married Brit who has moved from field work to training recruits as Season 2 opens, and Damien Scott (Sullivan Stapleton), a U.S. Delta Force operative, discharged on the eve of the U.S. invasion of Iraq, who made the most of his second chance to help Section 20 bring down an international terrorist last season. Again co-produced by Cinemax and the U.K.s Sky TV, the ten Season 2 episodes (47 min. each) follow Stonebridge, Scott and new boss Rachel Dalton (Rhona Mitra) as they scour Africa in pursuit of a fresh covey of criminals, each vying for possession of a box of valuable nuclear triggers that could launch Armageddon if they fall in the wrong hands.",Strike Back: Season 2,Various Artists
1397656,3,A1GF8W7J9526AB,B00LZ2LYQ6,Horror,"An aging caretaker for a powerful vampire must help hunt down a rogue half-breed vampire at the same time that he trains his two new apprentices. Bill Johnson is best know for playing LEATHERFACE in Texas Chainsaw Massacre Part 2. Joe Estevez, who is the brother to Martin Sheen and uncle to Charlie Sheen and Emilio Estevez has been in over 200 feature films including Apocalypse Now & Soul Taker.",The Caretakers,Nick Faust
1447797,1,A1GF8W7J9526AB,B00OHLR6KU,Science Fiction & Fantasy,"Trapped under a dark spell, Gretel assembles a coven of witches that starts to terrorize the town until Hansel gains the courage to fight his sister and the sinister forces that control her.",Hansel Vs Gretel,Brent Lydic
1521116,1,A1GF8W7J9526AB,B00SG82KEM,Science Fiction & Fantasy,"Six innocent women in Northern Italy were found guilty of witchcraft and burned at the stake during the Holy Inquisition. Satan s servant proposes to them the opportunity to avenge the injustice in exchange for their becoming his eternal immortal slaves. In acceptance; the women are brought back to life several centuries later as witches with evil powers that are used to destroy the Church and all who serve it. Father Don Gabriel, a courageous Priest and Exorcist, is sent to investigate numerous mysterious and ritualistic sacrifices in the countryside where the witches dwell. He is forced into the ultimate battle of Good vs. Evil; fighting for control of humanity.<P> <BR> <b>DVD Special Features</b>:<BR> -Behind-the-scenes Videos Barbara Bouchet's most memorable roles are the brazen spoiled rich lady ""Patrizia"" in Lucio Fulci's disturbing Don't Torture a Duckling (""Don't Torture A Duckling""), prostitute ""Francine"" in The French Sex Murders (""The French Sex Murders""), modeling agency choreographer ""Kitty"" in The Lady in Red Kills Seven Times (""Red Queen Kills 7 Times""), saucy love interest ""Scilla"" in the splendidly sleazy Ricco, and enticing stripper ""Anny"" in Death Rage (""Death Rage""). Bouchet had an unforgettably steamy lesbian love scene with Rosalba Neri in Amuck (""Amuck"") and appeared alongside fellow Bond girls Barbara Bach and Claudine Auger in Black Belly of the Tarantula (""The Black Belly of the Tarantula""). Barbara married producer Luigi Borgh

### Inferences for second recommendaion:
Based on the user's previous ratings and the genres they enjoyed, it seems that the recommendations lean towards drama and crime-related films or TV shows. Given the user's rating history, the content found in their user profile suggests that these recommendations might be suitable. However, it's important to note that the genres for some recommendations are labeled as "Unknown" but have been identified in the description.

# Conclusions
The evaluation of our initial model, SVD_first, showcased a slight improvement over the benchmark score:

**SVD_first RMSE**: 0.9106
**Benchmark RMSE**: 0.9352

To improve performance I'd like to do the following:
- Engage in further feature engineering for deeper insights and potential performance enhancement.
- Allocate additional resources to hyperparameter tuning for better-performing models and precise recommendations.
- Refine content-based filtering, as some suggestions had "Unknown" genres; tap into descriptive data for improvement.
By addressing these issues, the overall performance of the collaborative filtered recommendations should improve.